# Welcome to RunGPU Vision

This is a notebook to exclusively demonstrate the image model training capabilities that the Rungpu library offers. It's simple to use, has a plug-and-play interface. 

Image models can be used for several practices,but in this case we would be using Image models for training them to classify images 


In [1]:
from rungpu_vision import Model, Client, VisionDataset
from configparser import ConfigParser

### Enter your Rungpu Client_id and Client_Secret

In [2]:
config = ConfigParser()
config.read('system.ini')

client_id = config.get('rungpu_client','client_id')
client_secret = config.get('rungpu_client','client_secret')


## Create the Client Object for Authentication

In [3]:
client = Client(client_id, client_secret)

In [4]:
# Define a Dataset
dataset_config =  {"src_type": "kaggle", "src_id": "asdasdasasdas/garbage-classification"}
dataset = VisionDataset(client,dataset_config)



#### Create your Image dataset using the `create_dataset()` function

In [5]:
imgdat = dataset.create_dataset()
dataset_id = imgdat['dataset_id']
print(dataset_id)

imgdat_e90996f3-3b17-4a90-8ea6-2ed49a548917


#### Image Dataset Status

Since these are image model, its a safe bet to keep checking the status of the image dataset ingestion into the RunGPU Platform since some image datasets might take time to download. 

Also, The `check_data` gives you a `dir_map` feature, which let's you narrow out which specific directory has the right image-label mappings you would want to use for your classification model, and you can use that specific path in the `dataset_path` in the model configuration.

In [6]:
dataset.check_data(dataset_id)

{'Dataset ID': 'imgdat_e90996f3-3b17-4a90-8ea6-2ed49a548917',
 'dataset_status': {'command': 'Vision',
  'status': 'IN_PROGRESS',
  'dataset_id': 'imgdat_e90996f3-3b17-4a90-8ea6-2ed49a548917'}}

### Define the model training configuration

In [7]:
# Define a Config

config = {
    "model": "<name of the pretrained model you want to use>",
    # "dataset": "<path-for-the-dataset>"
    "n_classes": "<Number of classes",
    "optimizer" : "<optimizer-for-training>",
    "learning_rate" : "<learning rate used during training>",
    "momentum": "<momentum used during training>",
    "metrics" : "<metrics for training>",
    "epochs": "<number of epochs you want to train for>",
    "batch_size": "<int batch size for training>",
    "layers": [
    # Stack your layers up in this list in the following format
        #(layer_name: <name_of_the_layer, layer_type: <CONV2D | RELU | FC>, dim:(20,20,..)),
        #(layer_name: <name_of_the_layer, layer_type: <CONV2D | RELU | FC>, dim:(20,20,..))
    ]
        

}

In [8]:
config = {
        
        "model": "resnet50",
        "optimizer": "SGD",
        "n_classes": 6,
        "learning_rate": 0.001,
        "labels":  {"cardboard": 0, "glass": 1,"metal": 2, "paper": 3,"plastic": 4, "trash": 5},
        "dataset_id": dataset_id,
        "dataset_path": "garbage_classification/Garbage_classification",
        "momentum": 0.9,
        "metrics": ['accuracy'],
        "loss": "sparse_categorical_entropy",
        "epochs": 10, 
        "layers": [('lin1',"FC",(2048,1080)), ('sig1',"SIG"),('lin2',"FC",(1080,6)),('relu1',"RELU")], 
        "batch_size": 16
}

# Create the Model

In [9]:
model = Model(client=client,config=config)


## Run the Model 

In [10]:
model = model.train_model()
model

{'Dataset ID': 'imgdat_e90996f3-3b17-4a90-8ea6-2ed49a548917', 'dataset_status': {'command': 'Vision', 'status': 'IN_PROGRESS', 'dataset_id': 'imgdat_e90996f3-3b17-4a90-8ea6-2ed49a548917'}}


In [11]:
model

{'message': "Your dataset hasn't been created yet please try in some time."}

In [12]:
# Get your model id

model_id = model['train_id']
model_id

KeyError: 'train_id'

## Let's check the status of your running Job

In [ ]:
from rungpu_vision import VisionStatus

status = VisionStatus(client, model_id)


##### Call the `get_status()` method on the status object to get the current training status of the job

In [ ]:
status.get_status()


{'Train_Id': 'resnet50-a7465fe5-9d14-4af4-aff9-3be5f1a8b96f',
 'time_elapsed': 'None',
 'train_status': {'command': 'Vision',
  'status': 'IN QUEUE',
  'phase': 'TRAINING',
  'model_id': 'resnet50-a7465fe5-9d14-4af4-aff9-3be5f1a8b96f',
  'dataset_created': 'NO',
  'run_history': 'null',
  'dataset_id': 'imgdat_7882ad02-d373-4e2e-a4cd-937fe34b1c38',
  'run_start': None,
  'run_end': None,
  'error': 'Job Still in Queue',
  'process_id': 'No Process Assigned',
  'config': {'client_id': 'n6p7iWSrknJqkLIwX0PGi',
   'client_secret': '3b32447eab2155d1905c44dcf0506e5ceddf3827b',
   'model': 'resnet50',
   'dataset_id': 'imgdat_7882ad02-d373-4e2e-a4cd-937fe34b1c38',
   'dataset_path': 'garbage_classification/Garbage_classification',
   'n_classes': 6,
   'optimizer': 'SGD',
   'epochs': 10,
   'batch_size': 16,
   'learning_rate': 0.001,
   'momentum': 0.9,
   'loss': 'sparse_categorical_entropy',
   'metrics': ['accuracy'],
   'layers': [['lin1', 'FC', [2048, 1080]],
    ['sig1', 'SIG'],
    